In [1]:
from pathlib import Path
from functools import partial

import bw2data as bd

import sys
sys.path.append("../")

from bw_scenarios.importer import SDFImporter
from bw_scenarios.strategies import link_scenario_on_keys, separate_keys, replace_field

PROJECT_NAME = "bc24-hackathon-bike-project"
BIO_DB_NAME = "ecoinvent-3.10-biosphere"

bd.projects.set_current(PROJECT_NAME)
bd.databases

Databases dictionary with 3 object(s):
	bike_production_example
	biosphere3
	ecoinvent-3.10-biosphere

In [2]:
file_path = Path("sdf.xlsx")

# store the scenario data in a variable
importer = SDFImporter.from_excel(file_path)

In [3]:
print(bd.Database("biosphere3").get("349b29d1-3e58-4c66-98b9-9d1a076efd2e").id)
print(bd.Database("biosphere3").get("349b29d1-3e58-4c66-98b9-9d1a076efd2e"))

5240
'Carbon dioxide, fossil' (kilogram, None, ('air',))


In [4]:
bike_db = bd.Database("bike_production_example")
for act in bike_db:
    print(f"act: {act}")
    for exc in act.exchanges():
        print(f"exc: {exc}")
        print(f"exc['output']: {exc['output']}")
        print(f"exc['output'][1]: {exc['output'][1]}")
        print(f"exc['output'][0]: {exc['output'][0]}")
        print(f"get input: {bd.Database(exc['input'][0]).get(exc['input'][1])}")
        print(f"id input: {bd.Database(exc['input'][0]).get(exc['input'][1]).id}")
        print(f"get: {bd.Database(exc['output'][0]).get(exc['output'][1])}")
        print(f"id: {bd.Database(exc['output'][0]).get(exc['output'][1]).id}")
        print("---")

act: 'carbon fibre production' (kg, DE, None)
exc: Exchange: 237.3 MJ 'natural gas production' (MJ, NO, None) to 'carbon fibre production' (kg, DE, None)>
exc['output']: ('bike_production_example', 'cf')
exc['output'][1]: cf
exc['output'][0]: bike_production_example
get input: 'natural gas production' (MJ, NO, None)
id input: 4364
get: 'carbon fibre production' (kg, DE, None)
id: 4365
---
exc: Exchange: 26.6 kilogram 'Carbon dioxide, fossil' (kilogram, None, ('air',)) to 'carbon fibre production' (kg, DE, None)>
exc['output']: ('bike_production_example', 'cf')
exc['output'][1]: cf
exc['output'][0]: bike_production_example
get input: 'Carbon dioxide, fossil' (kilogram, None, ('air',))
id input: 1034
get: 'carbon fibre production' (kg, DE, None)
id: 4365
---
act: 'natural gas production' (MJ, NO, None)
act: 'bike production' (bike, DK, None)
exc: Exchange: 2.5 kg 'carbon fibre production' (kg, DE, None) to 'bike production' (bike, DK, None)>
exc['output']: ('bike_production_example', 'bi

In [5]:
exc['output']

('bike_production_example', 'bike')

In [6]:
exc['input']

('bike_production_example', 'cf')

In [7]:
bd.Database(exc['output'][0]).get(exc['output'][1]).id

4363

In [8]:
# drop duplicate columns
importer.apply_strategy(separate_keys)

# replace biosphere database name
importer.apply_strategy(partial(replace_field, field="to database", replace={"biosphere3": BIO_DB_NAME}))
importer.apply_strategy(partial(replace_field, field="from database", replace={"biosphere3": BIO_DB_NAME}))

# get the ids.
importer.apply_strategy(link_scenario_on_keys)


In [9]:
importer.unlinked

,from activity name,from reference product,from location,from categories,from database,to activity name,to reference product,to location,to categories,to database,...,default,regular case,change 1 sibling,change all sibling,change production,change production and sibling,from id,to id,from code,to code


In [10]:
importer.write()

In [11]:
from bw_scenarios.meta import scenarios
scenarios

Scenarios dictionary with 5 object(s):
	change 1 sibling
	change all sibling
	change production
	change production and sibling
	regular case

In [12]:
from bw_scenarios.scenario import Scenario
Scenario("regular case").load()


{4365: [(4365, 'production', 1.0),
  (4364, 'technosphere', 200.0),
  (4365, 'technosphere', 0.01),
  (5240, 'biosphere', 13.0)],
 4363: [(4365, 'technosphere', 2.0), (4365, 'technosphere', 0.5)]}

In [13]:
bd.projects.current

'bc24-hackathon-bike-project'

In [15]:
bd.projects.dir

PosixPath('/Users/bp45th/Library/Application Support/Brightway3/bc24-hackathon-bike-project.64c8d4d9')